# Atypical Quantum Maps, Robustness

In [2]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import tensorflow as tf
import random

from tqdm.notebook import tqdm

from loss_functions import ProbabilityMSE, channel_fidelity_loss
from optimization import ModelQuantumMap, Logger
from kraus_channels import KrausMap
from lindblad_channels import CompactLindbladMap
from spam import InitialState, POVM, SPAM, CorruptionMatrix
from utils import saver
from synthetic_data import generate_spam_data, generate_spam_benchmark, generate_map_data

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [3]:
def model_pipeline(channel_target = None, 
                   shots=1024, 
                   N_map = None, 
                   num_iter_spam=None, 
                   num_iter_map=None, 
                   verbose=False,
                   use_corruption_matrix=True,):
    # Make Benchmark
    #################################################################################
    n = 4
    d = 2**n
    
    spam_target = generate_spam_benchmark(n=n, c1=0.9, c2=0.8)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, shots=shots)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=N_map, shots=shots)
    #################################################################################

    # Fit Models
    #################################################################################
    if use_corruption_matrix:
        povm = CorruptionMatrix(d)
    else:
        povm = POVM(d)

    spam_model = SPAM(init = InitialState(d),
                      povm = povm,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(500, verbose = verbose)

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = num_iter_spam,
                     verbose = verbose,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = d**2,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = ProbabilityMSE(),
                                            loss_function_val = channel_fidelity_loss,
                                            sample_freq = 20),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = num_iter_map,
                N = 500,
                verbose = verbose,
                )
    #################################################################################
    model.optimizer = None
    spam_model.optimizer = None
    model.inputs = None
    model.targets = None
    model.inputs_val = None
    model.targets_val = None

    return model

## Fit Models

### Corruption Matrix

In [8]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 4
d = 2**n

channel_target1 = CompactLindbladMap(d, 1, 1, 0.1)
model1 = model_pipeline(channel_target1, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=False)

model2 = model_pipeline(channel_target1, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=False)

model3 = model_pipeline(channel_target1, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=False)

#channel_target2 = CompactLindbladMap(d, 16, 10000, 0.001)
#model4 = model_pipeline(channel_target2, 
#                        shots=1024, 
#                        N_map=10000-6**n, 
#                        num_iter_spam = 1000,
#                        num_iter_map = 2000,
#                        verbose=True)

#model5 = model_pipeline(channel_target2, 
#                        shots=1024, 
#                        N_map=10000-6**n, 
#                        num_iter_spam = 1000,
#                        num_iter_map = 2000,
#                        verbose=True)

#model6 = model_pipeline(channel_target2, 
#                        shots=1024, 
#                        N_map=10000-6**n, 
#                        num_iter_spam = 1000,
#                        num_iter_map = 2000,
#                        verbose=True)



#saver([[channel_target1, channel_target2],
#       [model1, model2, model3, model4, model5, model6]], "data/"+"atypical_maps_robustness.model")

saver([[channel_target1],
       [model1, model2, model3]], "data/"+"atypical_maps_robustness.model")

0.01356805534246124 -0.8729201106925852
0.014052458588686517 -0.8724832278520965
0.014087418677619975 -0.8688736096624493


In [5]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 4
d = 2**n

channel_target2 = CompactLindbladMap(d, 16, 10000, 0.001)
model1 = model_pipeline(channel_target2, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=False)

model2 = model_pipeline(channel_target2, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=False)

model3 = model_pipeline(channel_target2, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=False)

saver([[channel_target2],
       [model1, model2, model3]], "data/"+"atypical_maps_robustness2.model")

0.012637697744371998 -0.7465407585423098
0.013001184062003103 -0.7435060942926193


NameError: name 'channel_target1' is not defined

In [6]:
model3 = model_pipeline(channel_target2, 
                        shots=1024, 
                        N_map=10000-6**n, 
                        num_iter_spam = 1000,
                        num_iter_map = 2000,
                        verbose=False)

saver([[channel_target2],
       [model1, model2, model3]], "data/"+"atypical_maps_robustness2.model")

0.012815780028288825 -0.7458507960966557
